## Open set classification: transfer learning using same learning as the mercedes car-tagging model
Differences:
- smaller resnet (18)
- retrained on mercedes vs. non-mercedes dataset

### Mercedes vs. non-mercedes dataset
2 class dataset split into train,val,test. 
- Class 0 (mercedes) images taken from all Mercedes variants from s3://car-tagging-datasets/all-variants//variants-5. Test-complex images are augmented 6 times (using augmentation script) so that the showroom and complex images are balanced. A subset is taken for the final dataset.
- Class 1 (non-mercedes) subset from http://ai.stanford.edu/~jkrause/cars/car_dataset.html 

In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [2]:
# Define input image dimensions for resizing
height = 448
width = 448

# Define model hyperparameters
lr = 0.01
momentum = 0.9
T_0 = 225 # e.g. 899 / 4 (train_dataset_size / batch_size)
T_mult = 1
epochs = 15
batch_size = 4 # For both train and test sets

# Define number of layers for the ResNet neural network, select from [18, 34, 50 ,101, 152]
num_layers = 18

pretrained_weights = True
unfreeze_all_layers = False # i.e. Default: 'False', unfreezes last layer only for tuning

train_augmentation = False

In [3]:
# model = './model/resnet_sgd_cosineannealing_inference.py'
model = './model/resnet_sgd_cosineannealing_inference.py'


model_save_name = 'ResNet-cars-open-set'
model_save_path = 's3://custom-labels-console-eu-west-1-52749662e9/for-custom-labels/cars-open-set/sagemaker-pytorch-models/'
#model_code_save_path = 's3://car-tagging-datasets/sagemaker-models/trained-model-custom-code'

In [4]:
# Warm Restart Parameters:

bucket = None
saved_model_path = None

# Uncomment (if necessary) and amend the below paths for warm restarts using custom pre-trained weights

# bucket = 'custom-labels-console-eu-west-1-52749662e9'
# saved_model_id = 'resnet50-sgd-cosineannealing-448x448-2021-01-08-14-07-00-745'
# saved_model_path = 'for-custom-labels/train-dataset-augmented/sagemaker-pytorch-models/{}/output/model.tar.gz'.format(saved_model_id)

In [5]:
train = 's3://car-tagging-datasets/open-set/train/'
validation = 's3://car-tagging-datasets/open-set/validation/'
test = 's3://car-tagging-datasets/open-set/test/'

In [6]:
sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()

#Check the status of dataloader
estimator = PyTorch(entry_point=model,
                    role=role,
                    framework_version='1.4.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.g4dn.xlarge',
                    base_job_name=model_save_name,
                    output_path=model_save_path,
#                     code_location=custom_code_uri,
                    metric_definitions=[
                        {'Name': 'train:loss', 'Regex': 'train Loss: (.*?);'},
                        {'Name': 'train:acc', 'Regex': 'train Acc: (.*?);'},
                        {'Name': 'validation:loss', 'Regex': 'validation Loss: (.*?);'},
                        {'Name': 'validation:acc', 'Regex': 'validation Acc: (.*?);'},
                        {'Name': 'learning_rate', 'Regex': 'lr: (.*?);'},
                        {'Name': 'epoch', 'Regex': 'epoch: (.*?);'},
                        {'Name': 'train_f1_score', 'Regex': 'train Avg. F1 Score: (.*?);'},
                        {'Name': 'validation_f1_score', 'Regex': 'validation Avg. F1 Score: (.*?);'},
                        {'Name': 'test_f1_score', 'Regex': 'test Avg. F1 Score: (.*?);'},
                        {'Name': 'classification_report', 'Regex': 'classification_report: (.*?);'}
                    ],
                    hyperparameters={
                        'image-height': height,
                        'image-width': width,
                        'batch-size': batch_size,
                        'epochs': epochs,
                        'lr': lr,
                        'momentum': momentum,
                        'T_0': T_0,
                        'T_mult': T_mult,
                        'num-layers': num_layers,
                        'pretrained-weights': pretrained_weights,
                        's3-bucket': bucket,
                        'warm-restart': saved_model_path,
                        'unfreeze-all-layers': unfreeze_all_layers,
                        'train-augmentation': train_augmentation
                    })

In [7]:
estimator.fit({'train': train,
               'validation': validation,
               'test': test})

2021-06-24 13:14:06 Starting - Starting the training job...
2021-06-24 13:14:07 Starting - Launching requested ML instancesProfilerReport-1624540445: InProgress
...
2021-06-24 13:14:56 Starting - Preparing the instances for training.........
2021-06-24 13:16:31 Downloading - Downloading input data...
2021-06-24 13:17:01 Training - Downloading the training image......
2021-06-24 13:18:07 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-24 13:18:08,970 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-06-24 13:18:08,991 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-06-24 13:18:08,994 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-06-24 13:18:09,284 sagemaker-containers INFO     Module default_user_module_name does not provide a se